In [ ]:
import os
import numpy as np
import tarfile
import geopandas as gpd
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.mask import mask
import math

In [ ]:

def extract_files(source_directory, target_directory, clipped_directory, shapefile_path):
    for root, dirs, files in os.walk(source_directory):
        for name in files:
            if name.endswith('.tar'):
                tar_path = os.path.join(root, name)
                rasterfileName = name.split('.')[0]
                file_target_directory = os.path.join(target_directory, rasterfileName)
                os.makedirs(file_target_directory, exist_ok=True)

                with tarfile.open(tar_path, 'r') as tar:
                    tar.extractall(path=file_target_directory)
                print(f'Extracted {tar_path} to {file_target_directory}')

                process_files(file_target_directory, clipped_directory, rasterfileName, shapefile_path)

def clip_raster(raster_path, shapes):
    with rasterio.open(raster_path) as src:
        out_image, out_transform = mask(src, shapes, crop=True)
        out_meta = src.meta.copy()
        out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})

        return out_image, out_meta

def process_files(file_target_directory, clipped_directory, rasterfileName, shapefile_path):
    shp = gpd.read_file(shapefile_path)
    geo = shp.geometry
    bands = ["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B9", "B10", "B11"]

    for band in bands:
        input_path = os.path.join(file_target_directory, f"{rasterfileName}_{band}.TIF")
        temp_output_path = os.path.join(file_target_directory, f"temp_{rasterfileName}_{band}.TIF")

        with rasterio.open(input_path) as src:
            transform, width, height = calculate_default_transform(
                src.crs, 'EPSG:3826', src.width, src.height, *src.bounds)

            kwargs = src.meta.copy()
            kwargs.update({
                'crs': 'EPSG:3826',
                'transform': transform,
                'width': width,
                'height': height
            })

            with rasterio.open(temp_output_path, 'w', **kwargs) as dst:
                for i in range(1, src.count + 1):
                    reproject(
                        source=rasterio.band(src, i),
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs='EPSG:3826',
                        resampling=Resampling.nearest)


        clipped, meta = clip_raster(temp_output_path, geo)
        raster_clipped_folder = os.path.join(clipped_directory, rasterfileName)
        os.makedirs(raster_clipped_folder, exist_ok=True)
        output_path = os.path.join(raster_clipped_folder, f"{rasterfileName}_{band}.tif")
        with rasterio.open(output_path, "w", **meta) as dest:
            dest.write(clipped)

        os.remove(temp_output_path)

source_directory = 'data/zip'
target_directory = 'data/project/extracted_data'
clipped_directory = 'data/project/clipped'
shapefile_path = 'data/study_area/study_area_v3.shp'

extract_files(source_directory, target_directory, clipped_directory, shapefile_path)


In [ ]:
def calculate_ndvi_ndbi(r, nir, swir, th):
    with rasterio.open(r) as red_src:
        red = red_src.read(1).astype('float32')

    with rasterio.open(nir) as nir_src:
        nir = nir_src.read(1).astype('float32')

    with rasterio.open(swir) as swir_src:
        swir = swir_src.read(1).astype('float32')

    with rasterio.open(th) as th_src:
        th = th_src.read(1).astype('float32')

    ndvi = (nir - red) / (nir + red)
    ndbi = (swir - nir) / (swir + nir)
    lst = (1321.0789 / math.log((774.8853 / (0.0003342 * th + 0.1) ) + 1)) -273.15

    return ndvi, ndbi, lst

def save_tiff(data, file_path, meta):
    meta.update({"driver": "GTiff", "dtype": 'float32', "count": 1})
    with rasterio.open(file_path, "w", **meta) as dest:
        dest.write(data, 1)

def process_raster_data(clipped_directory):
    for folder_name in os.listdir(clipped_directory):
        folder_path = os.path.join(clipped_directory, folder_name)
        if os.path.isdir(folder_path):
            r = os.path.join(folder_path, f"{folder_name}_B4.tif")
            nir = os.path.join(folder_path, f"{folder_name}_B5.tif")
            swir = os.path.join(folder_path, f"{folder_name}_B6.tif")
            th = os.path.join(folder_path, f"{folder_name}_B10.tif")

            ndvi, ndbi, lst = calculate_ndvi_ndbi(r, nir, swir, th)

            with rasterio.open(r) as src:
                meta = src.meta.copy()

            save_tiff(ndvi, os.path.join(folder_path, f"{folder_name}_NDVI.tif"), meta)
            save_tiff(ndbi, os.path.join(folder_path, f"{folder_name}_NDBI.tif"), meta)
            save_tiff(lst, os.path.join(folder_path, f"{folder_name}_lst.tif"), meta)

clipped_directory = 'data/project/clipped'
process_raster_data(clipped_directory)
